## Лабораторна 7

In [1]:
pip install scikit-surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 8.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp310-cp310-linux_x86_64.whl size=2357275 sha256=2a66bbf7cd3308c937775d07c21f79a350fead479d17ab88d12f311d0c38a8e6
  Stored in directory: /root/.cache/pip/wheels/4b/3f/df/6acbf0a40397d9bf3ff97f582cc22fb9ce66adde75bc71fd54
Successfully built scikit-surprise


### Завдання 1

<p>1. Завантажте датасет для рецензій  (ml-100k) за допомогою бібліотеки
Surprise. </p>

In [2]:
from surprise import Dataset, accuracy
from surprise import SVD, KNNBasic
from surprise.model_selection import cross_validate, train_test_split, GridSearchCV
import pandas as pd
import numpy as np
from collections import defaultdict
from surprise import AlgoBase, Dataset, PredictionImpossible

In [3]:
data = Dataset.load_builtin('ml-100k')

Dataset ml-100k could not be found. Do you want to download it? [Y/n] y
Trying to download dataset from https://files.grouplens.org/datasets/movielens/ml-100k.zip...
Done! Dataset ml-100k has been saved to /root/.surprise_data/ml-100k


<p>2. Виведіть перші 5 рядків завантаженого датасету.</p>

In [4]:
all_ratings = data.raw_ratings
df = pd.DataFrame(all_ratings, columns=['user_id', 'item_id', 'rating', 'timestamp'])
print(df.head())

  user_id item_id  rating  timestamp
0     196     242     3.0  881250949
1     186     302     3.0  891717742
2      22     377     1.0  878887116
3     244      51     2.0  880606923
4     166     346     1.0  886397596


<p>3. Реалізуйте два алгоритми для рекомендаційної системи на основі цього
датасету. Можна вибрати будь-які алгоритми з бібліотеки Surprise.</p>

In [5]:
trainset, testset = train_test_split(data, test_size=0.25)

algo1 = SVD()
algo2 = KNNBasic()

<p>4. Використайте крос-валідацію для підбору оптимальних параметрів для
обох алгоритмів.</p>

<p>Для початку використаємо GridSearchCV.</p>

<p>Для SVD</p>

In [6]:
param_grid = {'n_epochs': [5, 10, 20], 'lr_all': [0.002, 0.005], 'reg_all': [0.4, 0.6]}
gs_svd = GridSearchCV(SVD, param_grid, measures=['mae'], cv=5)
gs_svd.fit(data)
print("Найкращі параметри для SVD:", gs_svd.best_params['mae'])

Найкращі параметри для SVD: {'n_epochs': 20, 'lr_all': 0.005, 'reg_all': 0.4}


<p>Для KNNBasic</p>

In [7]:
param_grid = {'k': [20, 40, 60], 'sim_options': {'name': ['msd', 'cosine'], 'user_based': [True, False]}}
gs_knn = GridSearchCV(KNNBasic, param_grid, measures=['mae'], cv=5)
gs_knn.fit(data)
print("Найкращі параметри для KNNBasic:", gs_knn.best_params['mae'])

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...


<p>5. Оберіть найкращий алгоритм на основі середньої абсолютної помилки
(MAE). </p>

In [8]:
best_knn = gs_knn.best_estimator['mae']
best_svd = gs_svd.best_estimator['mae']

results_knn = cross_validate(best_knn, data, measures=['mae'], cv=5, verbose=False)
results_svd = cross_validate(best_svd, data, measures=['mae'], cv=5, verbose=False)

mae_knn = results_knn['test_mae'].mean()
mae_svd = results_svd['test_mae'].mean()

print(f"MAE для KNNBasic: {mae_knn}")
print(f"MAE для SVD: {mae_svd}")

best_algo = best_knn if mae_knn < mae_svd else best_svd
print(f"Найкращий алгоритм: {'KNNBasic' if mae_knn < mae_svd else 'SVD'}")

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
MAE для KNNBasic: 0.7692157466494562
MAE для SVD: 0.7658733590811186
Найкращий алгоритм: SVD


<p>6. Виведи рекомендації (10 фільмів) для конкретного користувача.</p>

In [9]:
def get_top_n(predictions, n=10):
    top_n = defaultdict(list)

    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]
    return top_n
    
best_algo = gs_svd.best_estimator['mae']
trainset = data.build_full_trainset()
best_algo.fit(trainset)

testset = trainset.build_anti_testset()
predictions = best_algo.test(testset)
top_n = get_top_n(predictions, n=10)
user_id = '196'  # приклад користувача
print(f"Рекомендації для користувача {user_id}:")
for iid, est in top_n[user_id]:
    print(f"Фільм: {iid}, Очікуваний рейтинг: {est:.2f}")

Рекомендації для користувача 196:
Фільм: 408, Очікуваний рейтинг: 4.26
Фільм: 169, Очікуваний рейтинг: 4.23
Фільм: 318, Очікуваний рейтинг: 4.22
Фільм: 64, Очікуваний рейтинг: 4.19
Фільм: 114, Очікуваний рейтинг: 4.17
Фільм: 483, Очікуваний рейтинг: 4.16
Фільм: 12, Очікуваний рейтинг: 4.15
Фільм: 1449, Очікуваний рейтинг: 4.13
Фільм: 603, Очікуваний рейтинг: 4.13
Фільм: 178, Очікуваний рейтинг: 4.11


## Завдання 2

In [11]:
from surprise import Dataset, Reader, Trainset
from surprise.model_selection import train_test_split
data = Dataset.load_builtin('ml-100k')
trainset, testset = train_test_split(data, test_size=0.2)

In [12]:
import numpy as np
from surprise import AlgoBase, PredictionImpossible

class MyOwnAlgorithm(AlgoBase):
    def __init__(self):
        AlgoBase.__init__(self)

    def fit(self, trainset):
        AlgoBase.fit(self, trainset)

        self.global_mean = np.mean([r for (_, _, r) in self.trainset.all_ratings()])
        return self

    def estimate(self, u, i):
        if not self.trainset.knows_user(u) or not self.trainset.knows_item(i):
            raise PredictionImpossible("Користувач або елемент невідомий.")
        return self.global_mean

In [13]:
from surprise import accuracy
from surprise.model_selection import KFold

# Створюємо екземпляр нашого алгоритму
my_algo = MyOwnAlgorithm()

# Використовуємо перехресну перевірку
kf = KFold(n_splits=3)

for trainset, testset in kf.split(data):
    my_algo.fit(trainset)
    predictions = my_algo.test(testset)
    accuracy.mae(predictions)

MAE:  0.9432
MAE:  0.9466
MAE:  0.9444


In [15]:
user_id = str(17)
user_inner_id = my_algo.trainset.to_inner_uid(user_id)
# Створюємо список всіх фільмів
all_items = my_algo.trainset.all_items()
# Робимо прогнози для всіх фільмів для користувача 17
recommendations = []
for item in all_items:
    try:
        est = my_algo.estimate(user_inner_id, item)
        recommendations.append((item, est))
    except PredictionImpossible:
        continue
recommendations.sort(key=lambda x: x[1], reverse=True)
top_recommendations = [my_algo.trainset.to_raw_iid(item_id) for item_id, _ in recommendations[:10]] # Отримуємо ID фільмів
print("Топ 10 рекомендацій для користувача 17:", top_recommendations)

Топ 10 рекомендацій для користувача 17: ['403', '31', '1119', '96', '195', '565', '744', '471', '410', '181']


### Висновки
##### Ця система використовує простий метод, який обчислює середній рейтинг для всіх фільмів. Алгоритм демонструє початковий рівень точності, який можна використовувати як основу для подальших покращень. Для підвищення точності варто застосувати складніші методи розрахунку схожості між користувачами або елементами, такі як алгоритми на основі схожості cosine чи Pearson.